# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [19]:
import wikipedia
import numpy as np

In [20]:
def get_random_page():
    try:
        return wikipedia.random(1)
    except Exception:
        return get_random_page()
    
def get_page_by_name(name:str):
    try:
        return wikipedia.page(name)
    except wikipedia.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        return get_page_by_name(random_option)


####  Код ниже можнно не запускать, под ним ячейка со спиклованным объектом

In [29]:
lang_to_pages = {
    "zh": dict(),  #  китайский
    "ja": dict(),  #  японский
    "ko": dict(),  #  корейский
    "hi": dict(),  #  хинди
    "lv": dict(),  #  латышский
}
for lang in lang_to_pages:
    wikipedia.set_lang(lang)
    for _ in range(10):
        random_page = get_random_page()
        lang_to_pages[lang][random_page] = get_page_by_name(random_page).content

d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\HSE subj data\актуальные проблемы компьютерной лингвистики\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [2]:
import pickle

In [34]:
with open(r"data/lang_to_pages.pkl", "wb") as file:
    pickle.dump(lang_to_pages, file)

## Загрузка объекта

In [125]:
with open(r"data/lang_to_pages_COPY.pkl", "rb") as file:
    lang_to_pages = pickle.load(file)


### Чистка текстов

In [126]:
import re



class NormRule:
    def __init__(self, from_:str, to_:str) -> None:
        self.from_ = from_
        self.to_ = to_

    def apply_rule(self, text:str) -> str:
        return text.replace(self.from_, self.to_)


class RegNormRule(NormRule):

    def apply_rule(self, text: str) -> str:
        return re.sub(self.from_, self.to_, text)



class Normalizer:

    rules = (
        NormRule("==", ""),
        RegNormRule(r"\s+", " "),
        RegNormRule("・+", " "),
    )


    @classmethod
    def normalize(cls, text:str) -> str:
        for rule in cls.rules:
            text = rule.apply_rule(text)
        return text


In [127]:
for lang in lang_to_pages:
    lang_to_title = lang_to_pages[lang]
    for title in lang_to_title:
        lang_to_title[title] = Normalizer.normalize(lang_to_title[title])

In [128]:
import nltk
from nltk import sent_tokenize
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/smertlove/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [129]:
from random import choice

In [131]:
key = choice(list(lang_to_pages["ja"].keys()))
txt = lang_to_pages["ja"][key] #  ko hi lv

print(repr(txt))
print(len(sent_tokenize(txt, "japanese")))


'邵 疇（しょう ちゅう、235年 - 274年）は、中国三国時代の呉の政治家 武将。字は温伯。揚州会稽郡の人。 生涯 邵疇は会稽太守郭誕の功曹をつとめていた。 鳳凰3年（274年）、会稽郡にて、孫晧は既に亡くなっており、章安侯孫奮が天子になるであろうという妖言が流行った。臨海太守奚熙は郭誕に書簡を送り、国政を非難した。郭誕は、奚熙の国政を非難する書簡のことは上言したが、妖言については上言しなかったということで逮捕されると、気が動転して自らの無実を明らかにできなかった。邵疇は郭誕に謁見していった。「ただいますべては私の責任ということにいたしますので、太守さまにはご心配ご無用です。」邵疇はすぐさま役人のもとに出頭すると、妖言について上言しなかったのは自分の処置によるものであり、太守の罪ではないと、すすんで陳述した。役人は邵疇のいうところを上聞したが、孫晧の怒りは全く衰えなかった。邵疇は、このままでは郭誕も死を免れぬであろうことを慮り、自殺をしてみずからの罪を証らかにしようとし、死に臨み、次のような言葉を残した。「私は辺鄙後に成長し、政治教化の道には通じておりませんでしたが、由緒ある家柄だということで、本郡の役所に身を置くこととなり、同輩たちをとびこえて、役所の中でも高い地位をえたのでございます。ただ盛んなるご教化を宣揚するのに力を添えることも、人々の幸福な生活を計ってやることもできぬままでありました。ただいま妖言が横行して、国家を傷つけ綱紀を乱しておりますが、私は、人々ががやがやとしゃべり立てる言葉は、もともと事実ではないのだから、たとえ家ごとに語られ人ごとに詠われたとしても、心配する必要もないのだと考えました。天下と申しますものはいわば貴重な宝器でありますのに、身分のない者どもがそれについて勝手な取りざたをいたしたのであります。その内容の下劣さを疾んで、それがお耳に達するのは堪えられず、汚いものに蓋をして、それを書きとめることもせず、騒がしさもやがては沈静化し、自然に終息するであろうのを待とうと考えた次第でございます。私は愚かな心をもって一途に、つねづねこの旨を強く主張いたしてまいりました。それゆえ郭誕も、その考えるところをまげて、だまって私の意見に従われたのでございました。このたびの過ちは、実は私に出たことでございます。ここに謹んで、死罪も覚悟のうえで、ご関係の役

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/japanese/[0m

  Searched in:
    - '/home/smertlove/nltk_data'
    - '/home/smertlove/sandbox/hse/nlp_hw/venv/nltk_data'
    - '/home/smertlove/sandbox/hse/nlp_hw/venv/share/nltk_data'
    - '/home/smertlove/sandbox/hse/nlp_hw/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### NLTK не поддерживает такую экзотику, значит будем городить своё.

In [132]:
class Tokenizer:

    delimiter = "."

    @classmethod
    def tokenize(cls, text:str) -> list[str]: 
        return text.split(cls.delimiter)


class HieroglyphicTokenizer(Tokenizer):
    delimiter = "。"


class DevanagariTokenizer(Tokenizer):
    delimiter = "।"


class RegularTokenizer(Tokenizer):

    @classmethod
    def tokenize(cls, text: str) -> list[str]:
        return nltk.sent_tokenize(text)


lang_to_tokenizer : dict[str, Tokenizer]= {
    "zh": HieroglyphicTokenizer,
    "ja": HieroglyphicTokenizer,
    "ko": Tokenizer,
    "hi": DevanagariTokenizer,
    "lv": RegularTokenizer,
}

In [160]:
corpus = []

for lang in lang_to_pages:
    tokenizer = lang_to_tokenizer[lang]
    lang_to_title = lang_to_pages[lang]

    for title in lang_to_title:
        sents = tokenizer.tokenize(lang_to_title[title])
        
        for sent in sents:
            corpus.append((sent.strip(), lang))


In [170]:
import pandas as pd

In [171]:
df = pd.DataFrame(corpus, columns=("sentence", "language"))
grouped_df = df.groupby("language")

###  Загружаем модель

In [153]:
from sentence_transformers import SentenceTransformer

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [154]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/home/smertlove/sandbox/hse/nlp_hw/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [157]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [158]:
cluster = KMeans(n_clusters = len(lang_to_pages), max_iter = 500, algorithm = 'elkan')

In [159]:
sents = [elem[0] for elem in corpus]

X = np.zeros((len(sents), 768))
for i, sent in enumerate(sents):
    X[i] = model.encode(sent)

cluster.fit(X)
labels = np.array(cluster.labels_)+1

'布拉迪斯拉發施洛雲體育會（斯洛伐克語：ŠK Slovan Bratislava），簡稱施洛雲，是斯洛伐克的一家足球俱樂部，主場設在首都布拉迪斯拉發'

# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

## Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением. 

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова) 
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```
